In [118]:
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, cohen_kappa_score



from utils.nilc_nlp import *
from utils import dataset_setup
from utils.save_results import *
from utils.read_results import *

In [ ]:
def filter_elements_by_text(dataset_A, dataset_B, text_column='essay_text'):
    """
    Filtra os elementos do dataset A que possuem o mesmo `essay_text` presente no dataset B.
    
    Parameters:
        dataset_A (pd.DataFrame): Dataset maior com a coluna `essay_text`.
        dataset_B (pd.DataFrame): Dataset menor com a coluna `essay_text`.
        text_column (str): Nome da coluna que contém os textos nos dois datasets.
    
    Returns:
        pd.DataFrame: Subconjunto do dataset A com os textos encontrados no dataset B.
    """
    # Converte os textos do dataset B para um conjunto para busca rápida
    texts_in_B = set(dataset_B[text_column])
    
    # Filtra o dataset A onde o texto está presente no conjunto de textos do B
    filtered_A = dataset_A[dataset_A[text_column].isin(texts_in_B)]
    
    return filtered_A

In [105]:
# Sample Data Preprocessing
def process_y(y):
    # Remove brackets, split by space, and take the first 5 values
    y_processed = [list(map(int, label.strip("[]").split()[:5])) for label in y]
    return np.array(y_processed)

# Encoding class labels to integers
def encode_classes(y):
    encoders = [LabelEncoder() for _ in range(y.shape[1])]
    y_encoded = np.array([encoders[i].fit_transform(y[:, i]) for i in range(y.shape[1])]).T
    return y_encoded, encoders

In [106]:
# Map grades to normalized values
grade_mapping = {0: 0, 40: 1, 80: 2, 120: 3, 160: 4, 200: 5}

# Normalize y labels
def normalize_y(y):
    return np.vectorize(grade_mapping.get)(y)

In [ ]:
def decode_predictions(predictions):
    """
    Decode normalized predictions (0-5) to the original format (0, 40, 80, 120, 160, 200).
    
    Parameters:
        predictions (list or numpy array): A list or array of normalized predictions (values 0-5).
    
    Returns:
        numpy array: Decoded predictions in the original format (values 0, 40, 80, 120, 160, 200).
    """
    reverse_grade_mapping = {0: 0, 1: 40, 2: 80, 3: 120, 4: 160, 5: 200}
    return np.vectorize(reverse_grade_mapping.get)(predictions)

In [107]:
path_to = './data/'

In [108]:
"""
Setup Dataset
"""
dataset_name, dataset_code = dataset_setup.setup_dataset(3)
essays_dataset = dataset_setup.getDataset(path_to, dataset_name)

Dataset escolhido: propor2024


In [109]:
model_name = "nilc_metrix"
experiment_name = "exp0"

In [ ]:
"""
Read NIlc Metrix dataset 
"""

dataset_name = "aes_enem_dataset"
path_to_save = create_experiment_folder(path_to, model_name, experiment_name, dataset_name)
filename_to_save = build_filename_to_save(model_name, experiment_name, dataset_name, "0")
dataset_propor = read_csv(path_to_save, filename_to_save)

dataset_name = "essaysFullGrade"
path_to_save = create_experiment_folder(path_to, model_name, experiment_name, dataset_name)
filename_to_save = build_filename_to_save(model_name, experiment_name, dataset_name, "0")
dataset_full_grade = read_csv(path_to_save, filename_to_save)

In [ ]:
"""
Read Train, Test and Validation file
"""
dataset_base_path = "/data/Datasets/"

path_to_propor = dataset_base_path + "propor2024/"
train_dataset_propor = read_csv(path_to_save, "train")
validation_dataset_propor = read_csv(path_to_save, "validation")
test_dataset_propor = read_csv(path_to_save, "test")

path_to_dataset_full_grade = dataset_base_path + "fullGradeEnemEssays2024/"
train_dataset_full_grade = read_csv(path_to_save, "train")
validation_dataset_full_grade = read_csv(path_to_save, "train")
test_dataset_full_grade = read_csv(path_to_save, "test")

In [ ]:
test_propor = filter_elements_by_text(dataset_propor, test_dataset_propor)
test_full_grade = filter_elements_by_text(dataset_full_grade, test_dataset_full_grade)
test_dataset = pd.concat([test_propor, test_full_grade], ignore_index=True)

validation_propor = filter_elements_by_text(dataset_propor, validation_dataset_propor)
validation_full_grade = filter_elements_by_text(dataset_full_grade, validation_dataset_full_grade)
train_propor = filter_elements_by_text(dataset_propor, train_dataset_propor)
train_full_grade = filter_elements_by_text(dataset_full_grade, train_dataset_full_grade)
test_dataset = pd.concat([validation_propor, validation_full_grade, train_propor, train_full_grade], ignore_index=True)

In [111]:
X = dataset.iloc[:, 6:]
y = normalize_y(process_y(dataset["grades"]))

In [126]:
limit = 1155
X_train = X.iloc[:1100, :]
X_test = X.iloc[1100:, :]
y_train = y.iloc[:1100, :]
y_test = y.iloc[1100:, :]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42)

In [128]:
# Create and train a separate model for each competency
models = {}
predictions = {}
accuracies = []

best_params = {
    'colsample_bytree': 1.0,
    'learning_rate': 0.1,
    'max_depth': 7,
    'n_estimators': 50,
    'subsample': 0.6
}

for i in range(5):  # c1, c2, ..., c5
    print(f"Training model for c{i+1}...")
    
    # Extract labels for the current competency
    y_train_current = y_train[:, i]
    print(y_train_current)
    y_test_current = y_test[:, i]
    
    # Initialize XGBoost model
    model = xgb.XGBClassifier(
        objective='multi:softmax',  # Multiclass classification
        num_class=6,               # 6 classes: {0, 40, 80, 120, 160, 200}
        eval_metric='mlogloss',    # Multiclass log loss
        **best_params
    )
    
    # Train the model
    model.fit(X_train, y_train_current)
    
    # Store the model
    models[f"c{i+1}"] = model
    
    # Predict on the test set
    predictions[f"c{i+1}"] = model.predict(X_test)
    
    # Evaluate accuracy
    accuracy = accuracy_score(y_test_current, predictions[f"c{i+1}"])
    accuracies.append(accuracy)
    print(f"Accuracy for c{i+1}: {accuracy:.2f}")
    qwk = cohen_kappa_score(y_test_current, predictions[f"c{i+1}"],  weights="quadratic")
    print(f"QWK for c{i+1}: {qwk:.2f}")

# Combine all predictions
final_predictions = np.column_stack([predictions[f"c{i+1}"] for i in range(5)])
print("Final Predictions:\n", final_predictions)

Training model for c1...
[2 2 4 0 0 3 4 4 3 4 2 1 3 3 5 4 4 3 3 3 3 4 5 4 3 4 3 5 3 5 3 4 5 5 4 4 4
 4 3 2 4 4 3 3 3 5 3 4 4 3 2 3 3 3 4 5 3 0 3 5 3 4 0 4 3 4 5 0 2 3 4 2 3 3
 2 3 3 2 3 2 4 0 0 4 4 4 3 3 4 2 3 3 0 4 3 5 4 4 4 2 2 4 5 4 3 4 0 3 4 4 3
 5 5 4 0 4 2 2 2 4 4 5 3 0 3 3 0 2 3 2 4 3 3 3 3 4 2 3 4 3 2 5 5 4 3 3 4 4
 2 4 5 3 3 2 5 5 4 3 4 3 3 3 3 2 1 4 4 3 5 4 3 4 3 4 3 1 3 4 4 0 3 4 4 4 2
 4 2 4 3 1 4 4 3 3 4 2 4 2 2 3 5 5 4 5 3 4 3 4 3 4 4 2 3 5 3 4 3 4 3 4 4 5
 1 4 4 0 3 0 3 4 4 5 4 3 3 4 3 3 4 5 4 4 4 4 4 4 3 3 2 3 3 0 3 4 3 4 2 3 3
 4 3 2 2 4 4 3 2 3 3 4 1 5 3 3 4 4 5 4 5 2 4 4 4 3 3 3 3 4 4 2 4 4 4 5 0 3
 4 4 2 4 3 3 5 0 0 2 3 1 3 0 3 4 4 5 3 3 3 2 0 3 4 4 3 3 3 3 3 4 4 0 5 4 4
 0 3 2 4 4 2 5 5 4 4 3 3 3 4 4 4 4 4 3 3 4 3 3 3 3 3 5 4 4 3 0 4 4 3 3 4 5
 0 5 5 4 4 2 3 5 4 2 3 3 1 5 3 5 3 4 4 4 5 3 0 3 5 3 4 2 5 2 3 3 0 3 4 4 3
 3 0 2 3 3 0 3 4 5 4 0 3 3 3 5 4 4 4 3 3 4 4 5 3 2 4 4 4 3 3 0 3 5 3 3 0 4
 3 4 2 4 4 5 3 3 3 3 3 3 5 4 5 5 4 2 5 2 3 4 2 4 4 5 3 3 4 3 5 4 3 0 4 3 4


## Grid Search

In [129]:
# Define a custom scorer for Cohen's Kappa
def cohen_kappa_scorer(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")

kappa_scorer = make_scorer(cohen_kappa_scorer, greater_is_better=True)

# Define parameter grid
param_grid = {
    "learning_rate": [0.01, 0.05, 0.1],             # Finer tuning of learning rate
    "max_depth": [7, 9, 11],                         # Explore deeper trees
    "n_estimators": [50, 100, 200],                 # Increase the number of trees
    "subsample": [0.4, 0.6, 0.8],                   # Explore different subsets of data
    "colsample_bytree": [0.6, 0.8, 1.0],            # Column sampling
    "min_child_weight": [1, 3, 5],                  # Regularization for splits
    "gamma": [0, 1, 5],                             # Minimum loss reduction
    "scale_pos_weight": [1, 2, 3],                  # Class balancing
    "max_delta_step": [0, 1, 5]                     # Improves convergence for imbalanced data
}


In [130]:
xgb_model = xgb.XGBClassifier(objective="multi:softmax", num_class=6, eval_metric="mlogloss")

In [132]:
# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring=kappa_scorer,
    cv=3,  # 3-fold cross-validation
    verbose=2,
    n_jobs=-1
)

In [133]:
# Extract labels for the current competency
y_train_c1 = y_train[:, 0]
X_train_c1 = X_train

In [ ]:
# Fit GridSearchCV
grid_search.fit(X_train_c1, y_train_c1)

# Get the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best QWK Score:", grid_search.best_score_)

Fitting 3 folds for each of 19683 candidates, totalling 59049 fits


/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:36:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:36:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:36:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:36:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:36:27] WARNING: /workspace/src/le

[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=50, scale_pos_weight=1, subsample=0.6; total time=   4.9s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=50, scale_pos_weight=2, subsample=0.6; total time=   4.6s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=50, scale_pos_weight=3, subsample=0.6; total time=   5.3s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=100, scale_pos_weight=1, subsample=0.4; total time=   7.8s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=100, scale_pos_weight=2, subsample=0.4; total time=   6.7s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=

/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:42:24] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [15:42:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=50, scale_pos_weight=1, subsample=0.6; total time=   4.7s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=50, scale_pos_weight=2, subsample=0.6; total time=   4.6s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=50, scale_pos_weight=3, subsample=0.4; total time=   4.4s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=50, scale_pos_weight=3, subsample=0.8; total time=   6.5s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7, min_child_weight=1, n_estimators=100, scale_pos_weight=1, subsample=0.8; total time=  11.3s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=7

...

/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:38] WARNING: /workspace/src/le

[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=3, n_estimators=200, scale_pos_weight=1, subsample=0.6; total time=  10.2s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=3, n_estimators=200, scale_pos_weight=2, subsample=0.6; total time=   8.1s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=3, n_estimators=200, scale_pos_weight=3, subsample=0.6; total time=   7.5s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=5, n_estimators=50, scale_pos_weight=1, subsample=0.6; total time=   2.4s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=5, n_estimators=50, scale_pos_weight=2, subsample=0.4; total time=   1.8s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, mi

/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:51] WARNING: /workspace/src/le

[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=3, n_estimators=200, scale_pos_weight=2, subsample=0.6; total time=   9.0s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=3, n_estimators=200, scale_pos_weight=3, subsample=0.4; total time=   5.9s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=5, n_estimators=50, scale_pos_weight=1, subsample=0.4; total time=   1.1s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=5, n_estimators=50, scale_pos_weight=1, subsample=0.6; total time=   1.8s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min_child_weight=5, n_estimators=50, scale_pos_weight=1, subsample=0.8; total time=   1.8s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_delta_step=5, max_depth=9, min

/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/home/daniel/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [17:17:56] WARNING: /workspace/src/le